In [1]:
import pdfplumber
import re
import unidecode
import os

pdf_folder = r"D:\Biopark\4p\PI3\documents\raw"

def clean_text(text):
    """Limpa o texto extraído do PDF."""
    if not text:
        return ""
    
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'__', '', text)
    return text.strip()

def extract_entities(text):
    """Extrai CNPJs, CPFs, RGs e CEPs do texto."""
    cnpjs = re.findall(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', text)
    cpfs  = re.findall(r'\d{3}\.\d{3}\.\d{3}-\d{2}', text)
    rgs   = re.findall(r'\d{1,2}\.\d{3}\.\d{3}-\d{1}', text)
    ceps  = re.findall(r'\d{5}-\d{3}', text)
    return cnpjs + cpfs + rgs + ceps

all_chunks = []
global_chunk_number = 0
max_tokens = 400
overlap = 30

import tiktoken
encoding = tiktoken.get_encoding("cl100k_base")

def text_to_tokens(text):
    return encoding.encode(text)

def tokens_to_text(tokens):
    return encoding.decode(tokens)

def chunk_text_by_tokens(text, max_tokens, overlap):
    tokens = text_to_tokens(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokens_to_text(chunk_tokens)
        # Ignora chunks menores que o overlap, exceto se tiver entidades
        if len(chunk_tokens) >= overlap or extract_entities(chunk_text):
            chunks.append(chunk_text)
        start += max_tokens - overlap
    return chunks

# Processa PDFs
for pdf_file in os.listdir(pdf_folder):
    if not pdf_file.lower().endswith(".pdf"):
        continue
    pdf_path = os.path.join(pdf_folder, pdf_file)
    
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                page_text = clean_text(page_text)
                chunks = chunk_text_by_tokens(page_text, max_tokens=max_tokens, overlap=overlap)
                
                for chunk in chunks:
                    # chunk original
                    all_chunks.append({
                        "chunk_number": global_chunk_number,
                        "text": chunk,
                        "source_file": pdf_file,
                        "page_number": i
                    })
                    global_chunk_number += 1
                    
                    # chunk só com entidades (mesmo que seja menor que overlap)
                    entities = extract_entities(chunk)
                    if entities:
                        entity_text = " ".join(entities)
                        all_chunks.append({
                            "chunk_number": global_chunk_number,
                            "text": entity_text,
                            "source_file": pdf_file,
                            "page_number": i
                        })
                        global_chunk_number += 1

print(f"Total de chunks gerados: {len(all_chunks)}")
print("Exemplo de chunk com metadados:\n", all_chunks[0])


Total de chunks gerados: 64
Exemplo de chunk com metadados:
 {'chunk_number': 0, 'text': 'CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, ARS ARQUITETURA & PLANEJAMENTO S/S LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 01.980.174/0001-20, com sede na Rua Guarani, nº 2.106, Centro, no município de Toledo, Estado do Paraná, CEP 85.902-030, neste ato, representada por seu sócio administrador, Sr. Antonio Ricardo Nunes Sardo, brasileiro, casado, arquiteto, portador do RG n° 1008108514 SSP/RS, inscrito no CPF sob o nº 438.971.420-15, residente e domiciliado na Rua Padre Aloys Mann, n° 1.255, Conjunto Residencial Rossoni, no município de Toledo, Estado do Paraná, CEP 85.901-110, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.709/0001-03, com sede na Rodovia PR-182, Km 320/321, Condomín

In [2]:
import json
import os

# Pasta de destino
output_folder = r"D:\Biopark\4p\PI3\documents\processed"
os.makedirs(output_folder, exist_ok=True)

# Nome do arquivo de saída
output_file = os.path.join(output_folder, "chunks.json")

# Salvar todos os chunks
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=4)

print(f"✅ {len(all_chunks)} chunks salvos em {output_file}")

✅ 64 chunks salvos em D:\Biopark\4p\PI3\documents\processed\chunks.json


In [3]:
import json

input_file = r"D:\Biopark\4p\PI3\documents\processed\chunks.json"

with open(input_file, "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"Total de chunks carregados: {len(chunks)}")
print("Exemplo:", chunks[0])

Total de chunks carregados: 64
Exemplo: {'chunk_number': 0, 'text': 'CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, ARS ARQUITETURA & PLANEJAMENTO S/S LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 01.980.174/0001-20, com sede na Rua Guarani, nº 2.106, Centro, no município de Toledo, Estado do Paraná, CEP 85.902-030, neste ato, representada por seu sócio administrador, Sr. Antonio Ricardo Nunes Sardo, brasileiro, casado, arquiteto, portador do RG n° 1008108514 SSP/RS, inscrito no CPF sob o nº 438.971.420-15, residente e domiciliado na Rua Padre Aloys Mann, n° 1.255, Conjunto Residencial Rossoni, no município de Toledo, Estado do Paraná, CEP 85.901-110, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.709/0001-03, com sede na Rodovia PR-182, Km 320/321, Condomínio Industrial Biopark

In [4]:
from openai import OpenAI

from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()

# OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

embeddings_list = []

for chunk in chunks:
    response = client.embeddings.create(
        model="text-embedding-3-small",  # text-embedding-3-large ; text-embedding-3-small
        input=chunk["text"]
    )
    emb = response.data[0].embedding
    embeddings_list.append({
        "chunk_number": chunk["chunk_number"],
        "embedding": emb,
        "source_file": chunk["source_file"],
        "page_number": chunk["page_number"]
    })

print("✅ Embeddings gerados para todos os chunks!")

✅ Embeddings gerados para todos os chunks!


In [5]:
print(embeddings_list[0:10])

[{'chunk_number': 0, 'embedding': [0.01841825619339943, -0.008903230540454388, 0.02324821799993515, 0.0029945767018944025, -0.03071855939924717, 0.014801151119172573, -0.015734944492578506, -0.02715512178838253, -0.04168793931603432, -0.043469659984111786, 0.016690202057361603, -0.004974861163645983, 0.0213913656771183, -0.05431024357676506, 0.00398203544318676, 0.04387752339243889, -0.021659698337316513, -0.00700344517827034, 0.03460399806499481, -0.017087332904338837, 0.0026081795804202557, -0.009992655366659164, -0.01492995023727417, -0.05096146836876869, 0.007819171994924545, -0.04007795453071594, -0.038553833961486816, -0.0199101772159338, 0.023978078737854958, -0.04158060997724533, 0.02438594214618206, -0.01542367972433567, 0.00845780037343502, 0.03149135410785675, -0.056843288242816925, 0.04726923257112503, -0.027970848605036736, -0.027176586911082268, 0.0031314254738390446, -0.010223420336842537, 0.033165741711854935, -0.027541518211364746, 0.016250139102339745, 0.0396700911223

In [ ]:
import faiss
import numpy as np
import pickle

# Criar matriz com os embeddings
X = np.array([item["embedding"] for item in embeddings_list]).astype("float32")

for i, item in enumerate(embeddings_list):
    embeddings_list[i]["embedding"] = X[i]

# FAISS usa índices, podemos escolher L2 por padrão
index = faiss.IndexFlatL2(X.shape[1])
index.add(X)

# Salvar FAISS index
faiss.write_index(index, "embeddings.index")

# Salvar metadados separados
with open("metadata.pkl", "wb") as f:
    pickle.dump(embeddings_list, f)

print("✅ Embeddings e metadados salvos!")

✅ Embeddings e metadados salvos!


In [14]:
avaliacoes = [
    {"query": "Qual contrato tem a empresa com sede na Rua Alfred Charvet", "relevante": [0]},  # índices dos chunks relevantes
    {"query": "ANDREY BARRETO MOLINARI COMERCIO", "relevante": [32, 50]},
    {"query": "Qual empresa é representada pelo sócio administrador Jhonny Cézar de Jesus Falavinhacontrato", "relevante": [0]}, 
    {"query": "Jhonny Cézar de Jesus Falavinhacontrato", "relevante": [0, 30, 53]},
    {"query": "76.443.340/0001-59", "relevante": [52]}
]  

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

import numpy as np

def gerar_embedding(texto):
     
    resp = client.embeddings.create(
        model="text-embedding-3-large",
        input=texto
    )
    return np.array(resp.data[0].embedding, dtype="float32")
